In [136]:
import xmltodict
import subprocess
from glob import glob 
from pathlib import Path
from multiprocessing import Pool


In [153]:
cpp_field_template = "static constexpr bitfield_t<{reg_type},{start},{stop}> {name} = {{}};\n"
cpp_field_1bit_template = "static constexpr bitfield_t<{reg_type},{start}> {name} ={{}};\n"

cpp_reg_template = "static constexpr struct {reg_type}:reg_t<{str_type}, base_address+{offset}>\n\
{{\
{fields}\
using reg_t<{str_type}, base_address+{offset}>::operator=;\n\
using reg_t<{str_type}, base_address+{offset}>::operator|=;\n\
using reg_t<{str_type}, base_address+{offset}>::operator&=;\n\
}}{name}={{}};\n"

cpp_reg_no_field_template = "static constexpr reg_t<{str_type}, base_address+{offset}> {name}={{}};\n"

cpp_periph_template = "template<uint32_t base_address, int peripheral_index>\n struct {name}_t\n\
{{\n using tag = mst::tags::{tag};\n static constexpr int index = peripheral_index;\n static constexpr int address = base_address; \n {registers} }};\n"

c_reg_template = "volatile {reg_type} {name};\n"
c_periph_template = "struct {name}_c_t\n{{\n {registers} }};\n"

specific_vector_table_template = """
typedef void (*vector_table_entry_t)(void);

__attribute__ ((section(".specific_interrupt_vector_table"))) vector_table_entry_t specific_interrupt_vector_table[ {max_interrupt_count} ] =
{{
{vector_init}
}};
"""

In [154]:
MAX_INTERRUPT_COUNT = {'cm0': 32, 'cm3': 240, 'cm4': 240, 'cm7': 240}


def prnt(item, indent):
    if hasattr(item, 'prnt'):
        return item.prnt(indent)
    elif type(item) is dict:
        return "".join([
            (" " * indent) + name + ": " + prnt(subitem, indent + 2) + "\n"
            for name, subitem in item.items()
        ])
    else:
        return str(item)


class SVDElement:
    def __init__(self, dic, parse_table):
        for name, value in dic.items():
            if type(name) is str:
                self.__dict__[name.replace('@', '')] = parse_table.get(
                    name, lambda s, p: str(s))(value, parse_table)

    def prnt(self, indent=0):
        return "".join([
            (" " * indent) + name + ": " + prnt(item, indent + 2) + '\n'
            for name, item in self.__dict__.items()
        ])

    def __repr__(self):
        return self.prnt()


class Field(SVDElement):
    def __init__(self, field, parse_table):
        super(Field, self).__init__(field, parse_table)

    def to_cpp(self, reg_type):
        start = self.bitOffset
        stop = start + self.bitWidth - 1
        if self.bitWidth == 32:
            print(start, stop, self.name)
        if start != stop:
            return cpp_field_template.format(start=start,
                                             stop=stop,
                                             reg_type=reg_type,
                                             name=self.name)
        return cpp_field_1bit_template.format(start=start,
                                              reg_type=reg_type,
                                              name=self.name)


class Interrupt(SVDElement):
    def __init__(self, interrupt, parse_table):
        super(Interrupt, self).__init__(interrupt, parse_table)

    def __eq__(self, other):
        return self.name == other.name


class Cpu(SVDElement):
    def __init__(self, cpu, parse_table):
        super(Cpu, self).__init__(cpu, parse_table)

    def __eq__(self, other):
        return self.name == other.name


class Register(SVDElement):
    def __init__(self, reg, parse_table):
        super(Register, self).__init__(reg, parse_table)

    def __eq__(self, other):
        return self.name == other.name

    def to_cpp(self):
        name = self.name
        reg_type = self.name + "_t"
        offset = self.addressOffset
        if self.size == 32:
            str_type = 'uint32_t'
        elif self.size == 16:
            str_type = 'uint16_t'
        if len(self.fields) == 1 and next(iter(
                self.fields.values())).bitWidth == self.size:
            return cpp_reg_no_field_template.format(str_type=str_type,
                                                    offset=offset,
                                                    name=name)
        fields = "".join(
            [bitfield.to_cpp(reg_type) for bitfield in self.fields.values()])
        return cpp_reg_template.format(reg_type=reg_type,
                                       str_type=str_type,
                                       offset=offset,
                                       name=name,
                                       fields=fields)

    def to_c(self):
        name = self.name
        offset = self.addressOffset
        if self.size == 32:
            str_type = 'uint32_t'
        elif self.size == 16:
            str_type = 'uint16_t'
        return c_reg_template.format(reg_type=str_type, name=name)


class Peripheral(SVDElement):
    def __init__(self, periph, parse_table):
        super(Peripheral, self).__init__(periph, parse_table)

    def __eq__(self, other):
        return self.registers.keys() == other.registers.keys()

    def to_cpp(self, tag, name=None):
        if not name:
            name = self.name
        registers = "".join(
            [register.to_cpp() for register in self.registers.values()])
        return cpp_periph_template.format(name=name,
                                          registers=registers,
                                          tag=tag)

    def to_c(self, name=None):
        if not name:
            name = self.name
        offset = 0
        registers = ""
        for register in self.registers.values():
            while offset < register.addressOffset:
                registers += c_reg_template.format(reg_type="uint32_t",
                                                   name="__dummy" +
                                                   str(offset))
                offset += 4
            registers += register.to_c()
            offset += int(register.size / 8)
        return c_periph_template.format(name=name, registers=registers)


class Device(SVDElement):
    def __init__(self, dev, parse_table):
        super(Device, self).__init__(dev, parse_table)


_ctors_parse_table = {
    'field': Field,
    'register': Register,
    'peripheral': Peripheral,
}


def listify(item_or_list):
    if type(item_or_list) is list:
        return item_or_list
    else:
        return [item_or_list]


def ctor(key, dic, parse_table):
    return _ctors_parse_table[key](dic[key], parse_table)


def clang_format(fname: str):
    subprocess.run(["clang-format", "-i", '-style=file', fname])


_parse_table = {
    'addressOffset': lambda s, p: int(s, 16),
    'bitOffset': lambda s, p: int(s),
    'bitWidth': lambda s, p: int(s),
    'size': lambda s, p: int(s, 16),
    'resetValue': lambda s, p: int(s, 16),
    'cpu': lambda s, p: Cpu(s, p),
    'fields': lambda item, parse_table: {
        field['name']: Field(field, parse_table)
        for field in listify(item["field"])
    },
    'registers': lambda item, parse_table: {
        register['name']: Register(register, parse_table)
        for register in listify(item["register"])
    },
    'peripherals': lambda item, parse_table: {
        peripheral['name']: Peripheral(peripheral, parse_table)
        for peripheral in listify(item["peripheral"])
    },
    'interrupt': lambda item, parse_table: {
        interrupt['name']: Interrupt(interrupt, parse_table)
        for interrupt in listify(item)
    },
}


def make_specific_interrupt_vector(path: str, device: Device):
    interrupts = sorted({
        int(interrupt.value): name
        for periph in device.peripherals.values()
        if hasattr(periph, 'interrupt')
        for name, interrupt in periph.interrupt.items()
    }.items())

    vector_init = ",\n".join(
        [f'\t[{key}] __mst__isr_{name}' for key, name in interrupts])
    fname = f'{path}/specific_interrupt_vector_table.c'
    with open(fname, 'w') as f:
        f.write(
            specific_vector_table_template.format(
                max_interrupt_count=MAX_INTERRUPT_COUNT[
                    device.cpu.name.lower()],
                vector_init=vector_init))
        clang_format(fname)


#clang-format -i -style=file  gpio-regs.hpp
def make_header(path: str, peripheral: Peripheral, tag, name=None):
    if not name:
        name = peripheral.name
    cpp = peripheral.to_cpp(tag, name)
    c = peripheral.to_c(name)
    fname = f'{path}/{name.lower()}-regs.hpp'
    with open(fname, 'w') as f:
        f.write(f"""
    #pragma once
    #include <stdint.h>
    #include "mst/register.hpp"
    #include "mst/peripherals_tags.hpp"


    using namespace mst::registers;
    namespace mst::stm32::{name.lower()} {{
    {cpp}

    {c}
    }}
        """)

    clang_format(fname)


def load_svd(path: str):
    with open(path, 'r') as xf:
        desc = xmltodict.parse(xf.read())
        d = Device(desc['device'], _parse_table)
        for p in d.peripherals.values():
            if hasattr(p, 'derivedFrom'):
                p.__dict__['registers'] = d.peripherals[
                    p.derivedFrom].registers
        return d


def generate_all_sources(svd: str):
    device = load_svd(svd)
    path = device.description
    Path(path).mkdir(parents=True, exist_ok=True)
    make_specific_interrupt_vector(path, device)
    make_header(path, d.peripherals["RCC"], "no_tag")
    make_header(path, d.peripherals["GPIOA"], "gpio_tag", "gpio")
    make_header(path, d.peripherals["SDMMC1"], "sdmmc_tag", "sdmmc")
    make_header(path, d.peripherals["SPI1"], "spi_tag", "spi")
    make_header(path, d.peripherals["PWR"], "pwr_tag", "pwr")
    make_header(path, d.peripherals["SCB"], "scb_tag", "scb")
    make_header(path, d.peripherals["DMA1"], "dma_tag", "dma")
    make_header(path, d.peripherals["FLASH"], "flash_tag", "flash")

In [155]:
SVDs = glob('../../cpu/stm32/SVDs/**/*.svd', recursive=True)
with Pool(10) as p:
    p.map(generate_all_sources,SVDs)

In [125]:

d=load_svd("../../cpu/stm32/SVDs/STM32F7_svd/STM32F7_svd_V1.4/STM32F7x6.svd")

In [6]:
def all_same(device, periph_name):
    periphs = [p for n,p in device.peripherals.items() if periph_name in n]
    return all([[periph]*len(periphs)==periphs for periph in periphs])

all_same(d,'GPIO')
all_same(d,'SPI')

False

In [132]:
d.description

'STM32F7x6'

In [118]:
make_specific_interrupt_vector('.',d)

In [123]:
make_header('.',d.peripherals["RCC"], "no_tag")
make_header('.',d.peripherals["GPIOA"],"gpio_tag","gpio")
make_header('.',d.peripherals["SDMMC1"],"sdmmc_tag", "sdmmc")
make_header('.',d.peripherals["SPI1"],"spi_tag", "spi")
make_header('.',d.peripherals["PWR"],"pwr_tag", "pwr")
make_header('.',d.peripherals["SCB"],"scb_tag", "scb")
make_header('.',d.peripherals["DMA1"],"dma_tag", "dma")
make_header('.',d.peripherals["FLASH"],"flash_tag", "flash")

In [ ]:
rcc = d.peripherals["RCC"].to_cpp()
rcc_c = d.peripherals["RCC"].to_c()
with open('rcc-regs.hpp', 'w') as f:
    f.write(f"""
#pragma once
#include <stdint.h>
#include "mst/register.hpp"
#include "mst/peripherals_tags.hpp"

using namespace mst::registers;
namespace mst::stm32::rcc {{
{rcc}

{rcc_c }
}}
    """)

In [ ]:
gpio = d.peripherals["GPIOA"].to_cpp("GPIO")
gpio_c = d.peripherals["GPIOA"].to_c()
with open('gpio-regs.hpp', 'w') as f:
    f.write(f"""
#pragma once
#include <stdint.h>
#include "../register.hpp"


using namespace mst::registers;
namespace mst::stm32::gpio {{
{gpio}

{gpio_c }
}}
    """)

In [ ]:
d.peripherals.keys()

In [ ]:
sd = d.peripherals["SDMMC1"]

In [160]:
device.peripherals.keys()

dict_keys(['FSMC', 'PWR', 'RCC', 'GPIOA', 'GPIOB', 'GPIOC', 'GPIOD', 'GPIOE', 'GPIOF', 'GPIOG', 'AFIO', 'EXTI', 'DMA1', 'DMA2', 'RTC', 'BKP', 'IWDG', 'WWDG', 'TIM1', 'TIM2', 'TIM3', 'TIM4', 'TIM5', 'TIM12', 'TIM13', 'TIM14', 'TIM6', 'TIM7', 'I2C1', 'I2C2', 'SPI1', 'SPI2', 'SPI3', 'USART1', 'USART2', 'USART3', 'ADC1', 'DAC', 'DBG', 'UART4', 'UART5', 'CRC', 'FLASH', 'TIM15', 'TIM16', 'TIM17', 'CEC', 'NVIC', 'MPU', 'SCB_ACTRL', 'NVIC_STIR', 'SCB', 'STK'])